**IMPORT ZIP FILES TO EXTRACT DATA**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Python ML useful codes/archive.zip", 'r')
zip_ref.extractall("/content/drive/MyDrive/ML/Dataset/")
zip_ref.close()

In [ ]:
import os
size = 0
Folderpath = '/content/drive/MyDrive/ML/Helmet Detection/Dataset/val/labels/'
x= [(len(files)) for path, dirs, files in os.walk(Folderpath)]
print(x)

**INSTALLING REQUIRED LIBRARIES**

In [ ]:
!pip install ultralytics
!pip install yolov8
!pip install --upgrade pillow==6.2.2

**IMPORTING LIBRARIES**

In [ ]:
from ultralytics import YOLO
import os
import xml.etree.ElementTree as ET
import shutil
import random
!pip install tqdm --upgrade
from tqdm.notebook import tqdm


**DEFINING PATHS**

In [ ]:
train_path_img = "/content/drive/MyDrive/ML/Helmet Detection/NewDataset/train/images/"
train_path_label = "/content/drive/MyDrive/ML/Helmet Detection/NewDataset/train/Labels/"
val_path_img = "/content/drive/MyDrive/ML/Helmet Detection/NewDataset/val/images/"
val_path_label = "/content/drive/MyDrive/ML/Helmet Detection/NewDataset/val/Labels/"
test_path_img = "/content/drive/MyDrive/ML/Helmet Detection/NewDataset/test/images/"
test_path_label = "/content/drive/MyDrive/ML/Helmet Detection/NewDataset/test/Labels/"

**SPLITTING DATA INTO TRAIN TEST AND VAL**

In [ ]:
def train_test_split(path,neg_path=None, split = 0.1):
    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    val_size = int(len(files) * split)
    train_size = len(files) - test_size - val_size


    ## creating required directories
    #os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    #os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)
    #os.makedirs(test_path_img, exist_ok = True)
    os.makedirs(test_path_label, exist_ok = True)

    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      #shutil.copy2(path + filex + '.png',f"{train_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.xml', f"{train_path_label}/" + filex + '.xml')

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".png", f"{train_path_img}/" + filex + '.png')

    ### copytin images to validation folder
    for filex in tqdm(files[train_size:4500]):
      if filex == 'classes':
          continue
      # print("running")
      #shutil.copy2(path + filex + '.png', f"{val_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.xml', f"{val_path_label}/" + filex + '.xml')

    ### copytin images to test folder
    for filex in tqdm(files[4500:5000]):
      if filex == 'classes':
          continue
      # print("running")
      #shutil.copy2(path + filex + '.png', f"{test_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.xml', f"{test_path_label}/" + filex + '.xml')

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/ML/Dataset/annotations/') ### without negative images

**FOR DELETING CONTENTS OF ANY DIRECTORY**

In [ ]:
import os, shutil
folder = '/content/drive/MyDrive/ML/Helmet Detection/Dataset/train/labels/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

**TAKING BOUNDING BOXES, NORMALIZING BOUNDING BOXES AND STORING IN THEIR RESPECTIVE NAMES WITH .TXT**


In [ ]:
import os
import xml.etree.ElementTree as ET
import numpy as np

input_directory = '/content/drive/MyDrive/ML/Dataset/ValLabels/'
output_directory = '/content/drive/MyDrive/ML/Helmet Detection/Dataset/val/labels/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

def normalize_coordinateX(value1,value2, image_size):
    dw = np.float32(1. / int(image_size))
    dh = np.float32(1. / int(image_size))
    x = value1 + (value2 / 2)
    x = x * dw
    return x

def normalize_coordinateY(value1,value2, image_size):
    dw = np.float32(1. / int(image_size))
    dh = np.float32(1. / int(image_size))
    y = value1 + (value2 / 2)
    y = y * dw
    return x

def normalize_coordinateW(value1,value2, image_size):
    dw = np.float32(1. / int(image_size))
    dh = np.float32(1. / int(image_size))
    w = value2 - value1
    w = w * dh
    return w

def normalize_coordinateH(value1,value2, image_size):
    dw = np.float32(1. / int(image_size))
    dh = np.float32(1. / int(image_size))
    h = value2 - value1
    h = h * dh
    return h

def remap_class_label(original_label):
    class_mapping = {
        'helmet': 0,
        'head': 1,
        'person': 2,
    }
    return class_mapping.get(original_label, -1)

def extract_bounding_boxes(xml_file_path, image_width, image_height):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    bounding_boxes = []
    for labels in root.iter('object'):
        xmin = int(labels.find('bndbox/xmin').text)
        ymin = int(labels.find('bndbox/ymin').text)
        xmax = int(labels.find('bndbox/xmax').text)
        ymax = int(labels.find('bndbox/ymax').text)

        name_element = labels.find('name')
        if name_element is None:
            print("WARNING: Ignoring corrupt image/label: Class name not found")
            continue

        original_label = name_element.text
        new_label = remap_class_label(original_label)

        if new_label == -1:
            print(f"WARNING: Ignoring corrupt image/label: Invalid class label {original_label}")
            continue

        # Nomalize the coordinates between 0 and 1
        xmax_normalized = normalize_coordinateW(xmin,xmax, image_width)   #w
        ymax_normalized = normalize_coordinateH(ymin,ymax, image_height)  #h
        xmin_normalized = normalize_coordinateX(xmin,xmax_normalized, image_width)
        ymin_normalized = normalize_coordinateY(ymin,ymax_normalized, image_height)



        bounding_boxes.append([new_label, xmin_normalized, ymin_normalized, xmax_normalized, ymax_normalized])
        #bounding_boxes.append([new_label, xmin, ymin, xmax, ymax])

    return bounding_boxes

for file in os.listdir(input_directory):
    if file.endswith('.xml'):
        xml_file_path = os.path.join(input_directory, file)

        image_width = 416
        image_height = 416

        bounding_boxes = extract_bounding_boxes(xml_file_path, image_width, image_height)

        # Write bounding boxes to a new file in the output directory
        output_file_path = os.path.join(output_directory, os.path.splitext(file)[0] + '.txt')
        with open(output_file_path, 'w') as f:
            for box in bounding_boxes:
                f.write(' '.join(str(coord) for coord in box) + '\n')

print("Bounding boxes extracted, normalized, and saved successfully!")


Bounding boxes extracted, normalized, and saved successfully!


**FOR CREDATING LABELS.CACHE FILE FOR TRAINING**

In [ ]:
import os
import torch
import numpy as np
dic = {}

for i in os.listdir('/content/drive/MyDrive/ML/Helmet Detection/Dataset/test/images/'):
  with open(f'/content/drive/MyDrive/ML/Helmet Detection/Dataset/test/labels/{i.split(".png")[0]}.txt') as annotation:
    p = []
    for j in annotation.readlines():
      arr = list(map(float,j.strip().split()))
      p.append(arr)
  dic[f'/content/drive/MyDrive/ML/Helmet Detection/Dataset/test/images{i}'] = [np.array(p, dtype=float), (416,416)]

torch.save(dic,'/content/drive/MyDrive/ML/Helmet Detection/Dataset/test/labels.cache')

**PRETRAINED MODEL**

In [ ]:
model = YOLO('/content/drive/MyDrive/ML/Helmet Detection/Dataset/yolov8n.pt')

**TRAINING THE DATASET**

In [ ]:
results = model.train(
   data='/content/drive/MyDrive/ML/Helmet Detection/Dataset/data.yaml',
   imgsz=640,
   epochs=20,
   batch=32,
   lr0=0.01,
   name='/content/drive/MyDrive/ML/Helmet Detection/Training results/Helmet'
)


**VALIDATING DATASET**

In [ ]:
metrics = model.val()

**WEIGHTS MODEL**

In [ ]:
model = YOLO('/content/drive/MyDrive/ML/Helmet Detection/Training results/Helmet_10e/weights/best.pt')

**PREDICTING THE RESULTS*

In [ ]:
!yolo task = detect mode = predict model= model show = True conf =0.5 source = '/content/drive/MyDrive/ML/Helmet Detection/Dataset/test/images/hard_hat_workers1039.png'

In [ ]:
path = '/content/iii.jpeg'
result = model.predict(path, save=True)

**PLOTTING IMAGE WITH DETECTIONS USING MATPLOTLIB**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
#img = cv2.imread('/content/drive/MyDrive/ML/Helmet Detection/Dataset/test/images/hard_hat_workers1039.png')
#img = np.asarray(img)
#fig = plt.figure()
ax = plt.imshow(pred_in_array)


**TRACKING THE STREAMING**



In [ ]:
!pip install torchvision==0.14

In [ ]:
!pip install bytetracker

In [ ]:
!cp -r '/content/runs/detect/track3/demo_bytetrack.avi' '/content/drive/MyDrive/ML/Helmet Detection'    #transfering to new directory

In [ ]:
video_tracking = model.track('/content/demo.mp4', save = True, conf = 0.45, iou=0.55, persist = True,tracker="bytetrack.yaml")